# Power Analysis

Using a 2factor ANOVA, what sample size do we need to achive $\beta=0.8$?

> power is simply the proportion of times that we are able to reject the null hypothesis (remembering that we control the population means and we know that there is a true difference)

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set_style("whitegrid",{'axes.spines.left' : False,
                           'axes.spines.right': False,
                           'axes.spines.top': False,
                           'grid.linestyle': ':'})
sns.set_context("talk")
fp = "../data/artifacts/power_simulation-alpha=0.05.csv"

In [14]:
power_sim = pd.read_csv(fp)
power_sim.head()

,sample_size,power,avg_eta_sqr
0,15,0.100000,0.146470
1,15,0.166667,0.428379
2,15,0.033333,0.514556
3,15,0.033333,0.714745
4,15,0.100000,0.863468


In [ ]:
g = sns.lineplot(data=power_sim,
                 x="sample_size",
                 y="power")
thresh = power_sim.groupby("sample_size")["power"].mean()
sample_size = thresh[thresh >= 0.8].head(1).index[0]
g.set(xlabel="Sample Size",
      ylabel="Power")
# g.axhline(y=0.8,ls=':',c='red',linewidth=0.5)
g.axvline(x=sample_size,c='red',linewidth=0.5)
g.text(sample_size-10,
       0.83,
       f"Min Sample: {sample_size}",
       color='red',
       fontsize="small",
       horizontalalignment="right")
g.figure.savefig("../figures/artifacts/power_simulation-alpha=0.05-2.pdf")

## Simulation

In [ ]:
participants = pd.read_csv("../data/processed/mock_data/participant-schema.csv",index_col=0)
posts = pd.read_csv("../data/processed/mock_data/posts-schema.csv",index_col=0)
group_cols = ["PROLIFIC_ID","SESSION_ID","treatment","warning","evidence","code"]
reshare_rates = pd.DataFrame(posts.groupby(group_cols)["reshared"].sum() / posts.groupby(group_cols)["reshared"].count()).reset_index()
# Nicely format labels
reshare_rates['treatment'] = reshare_rates['treatment'].replace({False: 'Control',
                                                                 True: 'Treatment'})
reshare_rates['code'] = reshare_rates['code'].str.title()
reshare_rates["evidence"] = reshare_rates["evidence"].str.title()
affirmation_rates = reshare_rates[reshare_rates["code"]=="Affirms"]
affirmation_rates

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from tqdm.notebook import tqdm

def get_effect(df):
    """
        Gets the effect size, and significance, 
        
        Effect size, eta squared, is derived from this pdf: https://stardock.cs.virginia.edu/empirical/resources/Brown28.pdf
    """
    moore_lm = ols('reshared ~ C(treatment)*C(evidence)',data=df).fit()
    table = sm.stats.anova_lm(moore_lm, typ=2)
    pval = table.iloc[2,3]
    eta_sqr = table.iloc[2,0]/table.iloc[3,0]
    return eta_sqr,pval

sample_sizes = np.linspace(15,300,10,dtype=int)
num_trials = 10
trial_size = 2

def power_simulation(df,sample_sizes,num_trials,trial_size,alpha=0.05):
    res = {"sample_size": [],
           "power": [],
           "avg_eta_sqr": []}
    for ss in tqdm(sample_sizes):
#         assert len(df) > ss, f"Sample size ({ss}) larger than dataframe ({len(df)})"
        effect_size = 0
        for _ in range(num_trials):
            num_rejections = 0
            for _ in range(trial_size):
                sampled_df = df.sample(n=ss)
                try:
                    eta_sqr, pval = get_effect(sampled_df)
                    effect_size += eta_sqr
                    if pval < alpha: num_rejections +=1
                except:
                    trial_size-=1
            res["sample_size"].append(ss)
            res["power"].append(num_rejections/trial_size)
            res["avg_eta_sqr"].append(effect_size/trial_size)
    res = pd.DataFrame(res)
    return pd.DataFrame(res)

power_sim = power_simulation(affirmation_rates,
                             sample_sizes,
                             num_trials,
                             trial_size)

In [ ]:
sample_sizes = np.linspace(15,300,100,dtype=int)
num_trials = 50
trial_size = 30

power_sim = power_simulation(affirmation_rates,
                             sample_sizes,
                             num_trials,
                             trial_size)
power_sim.to_csv(fp,index=False)